In [108]:
import numpy as np
import pandas as pd

In [109]:
# Define the KNN class
class KNN:
    def __init__(self, k=3, distance_metric='euclidean'):
        self.k = k
        self.distance_metric = distance_metric

    def fit(self, X, y):
        # TODO: Implement the fit method
        self.X = X
        self.y = y
        pass

    def predict(self, X):
        # TODO: Implement the predict method

        predictions = []

        for row in X:
            #if row in self.X:
            #    predictions.append([self.y[i] for i in range(len(self.y)) if np.allclose(y[i], row)][0])
            #else:
                #distances = [[self.compute_distance(row, self.X[i]), self.Y[i]] for i in range(len(self.X))]
                #orted_dis = sorted(distances)
                #topk = sorted_dis[:self.k]

                #classification = self.neighbor_majority(topk)

                #predictions.append(classification)
            distances = np.apply_along_axis(self.compute_distance, 1, self.X, row)
            dis_classifications = np.array([[distances[i], self.y[i]] for i in range(len(self.y))]) #np.concatenate((distances, self.y), axis=1)
            #sort_dis_class = np.sort(dis_classifications)
            #sort_dis_class = sorted(dis_classifications)
            sort_dis_class = dis_classifications[dis_classifications[:, 0].argsort()]

            topk = sort_dis_class[:self.k]
            classification = self.neighbor_majority(topk)

            predictions.append(classification)

        return np.array(predictions)

    def neighbor_majority(self, topk):
        #class_counts = Counter([classification for _, classification in topk])

        #most_frequent_classes = class_counts.most_common()

        #if len(most_frequent_classes) == 1:
        #    return most_frequent_classes[0][0]
        #else:
        #    if most_frequent_classes[0][1] > most_frequent_classes[1][1]:
        #        return most_frequent_classes[0][0]
        #    else:
        #        inv_sq_sums = {0:0, 1:0}
        #        for dis, classification in topk.items():
        #            inv_sq_sums[classification] += 1 / (dis ** 2)
        #            return 1 if inv_sq_sums[1] > inv_sq_sums[0] else 0
        class_counts = {1:0, 0:0}
        inv_sq_sums = {1:0, 0:0}
        #print(topk)
        for dis, classification in topk:
            class_counts[classification] += 1
            inv_sq_sums[classification] += 1 / (dis ** 2)

        if class_counts[1] != class_counts[0]:
            #print(f"Majority: {1 if class_counts[1] > class_counts[0] else 0}")
            return 1 if class_counts[1] > class_counts[0] else 0
        else:
            return 1 if inv_sq_sums[1] > inv_sq_sums[0] else 0
        


        

    def compute_distance(self, X1, X2):
        # TODO: Implement distance computation based on self.distance_metric
        # Hint: Use numpy operations for efficient computation

        return np.linalg.norm(X1 - X2)

In [110]:
# Define data preprocessing function
def preprocess_data(train_path, test_path):
    train_data = pd.read_csv(train_path)
    test_data = pd.read_csv(test_path)

    labels = train_data["Exited"].to_numpy(dtype=int)
    
    train_dropped = train_data.drop(columns=["id", "CustomerId", "Surname", "Exited"])
    test_dropped = test_data.drop(columns=["id", "CustomerId", "Surname"])

    train_encoded = pd.get_dummies(train_dropped, columns=["Gender", "Geography"], dtype=int)
    test_encoded = pd.get_dummies(test_dropped, columns=["Gender", "Geography"], dtype=int)

    for col_name in train_encoded.columns:
        max_val = max(train_encoded[col_name].max(), test_encoded[col_name].max())
        if max_val != 0:
            train_encoded[col_name] = train_encoded[col_name] / max_val
            test_encoded[col_name] = test_encoded[col_name] / max_val


    # TODO: Implement data preprocessing
    # Handle categorical variables, scale features, etc.
    return train_encoded.to_numpy(dtype=float), labels, test_encoded.to_numpy(dtype=float)

In [111]:
# Define cross-validation function
def cross_validate(X, y, knn, n_splits=5):
    # TODO: Implement cross-validation
    # Compute ROC AUC scores

    data_splits = np.array_split(X, n_splits)
    class_splits = np.array_split(y, n_splits)

    for i in range(n_splits):
        curr_test_data = data_splits[i]
        curr_test_classes = class_splits[i]

        
        curr_train_data = [data_splits[j] for j in range(n_splits) if j != i]
        curr_train_classes = [class_splits[i] for j in range(n_splits) if j != i]

        train = np.concatenate(curr_train_data)
        train_classes = np.concatenate(curr_train_classes)


        knn.fit(train, train_classes)

        test_prediction = knn.predict(curr_test_data)
        results = test_prediction == curr_test_classes
        num_right = sum([1 for t in results if t])
        num_wrong = sum([1 for t in results if not t])
        print(f"Num right: {num_right}. Num wrong: {num_wrong}")
        count = 0
        #for pred, actual in zip(test_prediction, curr_test_classes):
        #    print(f"Prediction: {pred}. Actual: {actual}.")
        #    if count == 100:
        #        break
        #    count += 1
    return "hi"



In [112]:
# Load and preprocess data
X, y, X_test = preprocess_data('data/train.csv', 'data/test.csv')

# Create and evaluate model
knn = KNN(k=20, distance_metric='euclidean')

# Perform cross-validation
cv_scores = cross_validate(X, y, knn)

print("Cross-validation scores:", cv_scores)

# TODO: hyperparamters tuning
#for k_value in range(1, 21):
#    print(f"Value for k {k_value}")
#    knn = KNN(k=k_value, distance_metric="euclidean")
#    cross_validate(X, y, knn)


# TODO: Train on full dataset with optimal hyperparameters and make predictions on test set
#knn = KNN(k=20, distance_metric="euclidean")
#knn.fit(X, y)
#test_predictions = knn.predict(X_test)

# Save test predictions
#pd.DataFrame({'id': pd.read_csv('data/test.csv')['id'], 'Exited': test_predictions}).to_csv('submissions.csv', index=False)

Num right: 2380. Num wrong: 620
Num right: 2418. Num wrong: 582
Num right: 2388. Num wrong: 612


KeyboardInterrupt: 